Install prereqs

In [99]:
!pip install "tensorflow==2.7"
!pip install transformers

ERROR: Could not install packages due to an EnvironmentError: [Errno 28] No space left on device

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [100]:
from transformers import BertTokenizer
import pandas as pd
import itertools
import nltk
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split

In [101]:
nltk.download('vader_lexicon')


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\14803\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [102]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

df = pd.read_csv("bert_news.csv").dropna()

pred_sentences = df['text'].tolist()
map(str,pred_sentences)

tokens = []
failures = []
for p in pred_sentences:
    tokens.append(tokenizer(p, padding=True)['input_ids'])

padded = pd.DataFrame(list(zip(*itertools.zip_longest(*tokens, fillvalue=0))))
pd.concat([df,padded], axis=1).to_csv("news_vectors.csv",index=False, sep=",",header=True)


In [103]:
from transformers import BertTokenizer
import pandas as pd
import itertools

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

df = pd.read_csv("bert_tweets.csv").dropna()

pred_sentences = df['text'].tolist()
map(str,pred_sentences)

tokens = []
failures = []
for p in pred_sentences:
    tokens.append(tokenizer(p, padding=True)['input_ids'])

padded = pd.DataFrame(list(zip(*itertools.zip_longest(*tokens, fillvalue=0))))
pd.concat([df,padded], axis=1).to_csv("tweet_vectors.csv",index=False, sep=",",header=True)


In [104]:
pd.concat([df,padded], axis=1).to_csv("news_vectors.csv",index=False, sep=",",header=True)


In [105]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

sent = []
for text in df['text']:
    s = sid.polarity_scores(text)
    sent.append(s['compound'])

df['sentiment'] = sent

In [106]:
X = pd.concat([df[['sentiment']],padded], axis = 1)
Y = df['label']

In [107]:
X_train, X_test, Y_train, Y_test = train_test_split(X.to_numpy(), Y.to_numpy(), test_size=0.2, random_state=27)
logReg = LogisticRegression()
logReg.fit(X_train,Y_train)

C:\Users\14803\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [108]:
predictions = logReg.predict(X_test)
accuracy = logReg.score(X_test, Y_test)
f1 = f1_score(predictions, Y_test, average='binary')
precision = precision_score(predictions, Y_test)
recall = recall_score(predictions, Y_test)

print('Accuracy:', accuracy)
print('Precision:',precision)
print('Recall:', recall)
print('F1 Score:',f1)

Accuracy: 0.5329667080598427
Precision: 0.9673972904504902
Recall: 0.5360390601159597
F1 Score: 0.6898366321080741


In [109]:
import pickle
pickle.dump(logReg,open('bert_logreg.sav','wb'))